In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
fnews=pd.read_csv('/kaggle/input/indian-financial-news-articles-20032020/IndianFinancialNews.csv')
nnews=pd.read_csv('/kaggle/input/india-headlines-news-dataset/india-news-headlines.csv')

In [ ]:
nnews.head()

In [ ]:
stockDetails=pd.read_csv('/kaggle/input/nifty50-stock-market-data/KOTAKBANK.csv')


In [ ]:
stockDetails.head()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(stockDetails['Close'].to_list()[:])
dftoproc=dftoproc[:]

In [ ]:
fnews=fnews.drop(columns=['Description'])

In [ ]:
dfnews=pd.DataFrame(columns=['Date','Title'])
uni=fnews['Date'].unique()

for each in uni:
    df=fnews[fnews['Date']==each]
    tmp=""
    for title in df['Title']:
        tmp=tmp+str(title)+". "
    dfnews=dfnews.append({'Date':each,'Title':tmp},ignore_index=True)


In [ ]:
dfnews['Date'].nunique()

In [ ]:
newsobj=pd.DataFrame(columns=['Date','Title'])

for each in range(dfnews['Date'].nunique()-3):
    tmp1=dfnews['Title'][each]
    tmp2=dfnews['Title'][each+1]
    tmp3=dfnews['Title'][each+2]
    tmp=tmp1+tmp2+tmp3
    newsobj=newsobj.append({'Date':dfnews['Date'][each],'Title':tmp},ignore_index=True)
    

In [ ]:
import tensorflow_hub as hub
elmo=hub.load("https://tfhub.dev/google/elmo/3")
newsobj.head(10)

In [ ]:
!pip install sentence_transformers

In [ ]:
news_list=newsobj['Title'].to_list()

In [ ]:
newsobj['Date'].unique()[-1]
def chng(dt):
    lst=dt.split(" ")
    datetime_object = datetime.datetime.strptime(lst[0], "%B")
    dateobj=datetime.datetime(int(lst[2].strip(",")),int(datetime_object.month),int(lst[1].strip(",")))
    return dateobj.strftime("%Y-%m-%d")
for i in range(newsobj['Date'].nunique()):
    newsobj['Date'][i]=chng(newsobj['Date'][i])
    

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [ ]:
#Comment after encoding and saving
# corpus_embeddings = embedder.encode(news_list, convert_to_tensor=True)
# newsobj['Embed']=corpus_embeddings
#comment this after saving embeddings and use the text file  by loading 
# torch.save(corpus_embeddings,'embed.txt')

In [ ]:
newsobj['Embed']=torch.load('/kaggle/input/elmoembedings/embed.txt')

In [ ]:
newsobj.head()

In [ ]:
stockDetails.iloc[:,:].shape

In [ ]:
dftoproc=pd.merge(stockDetails,newsobj,on='Date')
# dftoproc=stockDetails

In [ ]:
dftoproc.head()

In [ ]:
dftoproc=dftoproc.drop('Turnover',axis=1)
dftoproc=dftoproc.drop('Trades',axis=1)
dftoproc=dftoproc.drop('Deliverable Volume',axis=1)
dftoproc=dftoproc.drop('%Deliverble',axis=1)
dftoproc=dftoproc.drop('Series',axis=1)
dftoproc=dftoproc.drop('Symbol',axis=1)

In [ ]:
# for i in range(3123):
#     dftoproc['Close'][i]=int(dftoproc['Close'][i])/10

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dftoproc['Close'].to_list()[:])
dftoproc=dftoproc[:]

In [ ]:
chg=[]
for i in range(dftoproc.shape[0]-1):
    chg.append((float(dftoproc['Prev Close'][i+1])-float(dftoproc['Prev Close'][i]))/float(dftoproc['Prev Close'][i]))
chg.insert(0,0.0)
len(chg)
dftoproc['change']=chg

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
temp=dftoproc['Close'].to_numpy()
dftoproc['Close'] = scaler.fit_transform(temp.reshape(temp.shape[0],1))

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return X,y
x,y=split_sequence(dftoproc['Close'].to_list(), 60)

In [ ]:
dftoproc=dftoproc.iloc[60:,:]

In [ ]:
dftoproc.shape

In [ ]:
len(x)

In [ ]:
dftoproc.insert(2,'Series',x)

In [ ]:
dftoproc.shape[0]

In [ ]:
dftoproc=dftoproc.reset_index()

In [ ]:
embed=dftoproc['Embed'].apply(lambda x: [x[i].numpy() for i in range(len(x))])
embed=embed.to_numpy()
embed=np.array([np.array(i) for i in embed])

In [ ]:
embed.shape

In [ ]:
series=dftoproc['Series'].apply(lambda x: [np.array(x[i]) for i in range(len(x))])

In [ ]:
series=series.to_numpy()
series=np.array([np.array(i) for i in series])

In [ ]:
series

In [ ]:
series = np.reshape(series, (series.shape[0], series.shape[1], 1))

In [ ]:
series.shape

In [ ]:
targ=dftoproc['Close'].to_numpy()
targ=targ.reshape((targ.shape[0],1))

In [ ]:
targ.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(series, targ, test_size=0.2,shuffle=False,stratify=None)
X_train2, X_test2, y_train2, y_test2 = train_test_split(embed, targ, test_size=0.2)




In [ ]:
X_train2.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout,Concatenate,Flatten
from keras.layers.merge import concatenate

model1=Sequential()
model1.add(Input(shape=(768,)))
model1.add(Dense(300,activation='relu'))
model1.add(Dense(100,activation='relu'))
model1.add(Dense(30,activation='relu'))
model1.add(Dense(15,activation='relu'))

model2=Sequential()
model2.add(LSTM(35,input_shape=(60, 1),return_sequences=True))
model2.add(LSTM(20,return_sequences=True))
model2.add(LSTM(10,return_sequences=True))
model2.add(Flatten())
model2.add(Dense(5))

modelx=concatenate([model1.output,model2.output])
modelx=Dense(1,activation='relu')(modelx)

model=Model(inputs=[model1.input,model2.input],outputs=modelx)

model.compile(loss='mean_squared_error',optimizer='adam')
model.fit([X_train2,X_train],y_train,epochs=2,batch_size=1)


In [ ]:
model.evaluate([X_test2,X_test],y_test)

In [ ]:
ans=model.predict([X_test2,X_test])

In [ ]:
y_test

In [ ]:
fig,ax=plt.subplots(1,figsize=(25,5))
ax.plot(ans[:-350])
ax.plot(y_test[:-350])
plt.show()

# Extras

In [ ]:
# from tensorflow.keras.models import Sequential,Model
# from tensorflow.keras.layers import LSTM,Dense,Input,Dropout,Concatenate,Flatten
# from keras.layers.merge import concatenate

# model2=Sequential()
# model2.add(LSTM(50,return_sequences=True, input_shape= (X_train.shape[1], 1)))
# model2.add(LSTM(50,return_sequences=False))
# model2.add(Dense(25))
# model2.add(Dense(1))


# model2.compile(loss='mean_squared_error',optimizer='adam')
# model2.fit(X_train,y_train,epochs=2,batch_size=1)


In [ ]:
series=series.reshape((series.shape[0],series.shape[1],1))

In [ ]:
b=model2.evaluate(series,targ)


In [ ]:
targ=y_test.reshape((y_test.shape[0],1,1))

In [ ]:
ans=model2.predict(X_test)

In [ ]:
ans=ans.reshape((-1,ans.shape[0]))


In [ ]:
ans = scaler.inverse_transform(ans)


In [ ]:
targ=targ.reshape((-1,targ.shape[0]))

In [ ]:
targ=scaler.inverse_transform(targ)

In [ ]:
ans=ans.reshape((ans.shape[1],ans.shape[0]))
targ=targ.reshape((targ.shape[1],targ.shape[0]))

In [ ]:
for i in range(len(ans)):
    print(ans[i],targ[i])

In [ ]:
fig,ax=plt.subplots(2,figsize=(25,5))

ax[0].plot(ans[:50],label='pred')
ax[0].plot(targ[:50],label='orig')
plt.legend()
plt.show()

In [ ]:
# stockDetails=pd.read_csv('/kaggle/input/stock-market-india/FullDataCsv/HDFCBANK__EQ__NSE__NSE__MINUTE.csv')
# srs=stockDetails['close'].dropna()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
temp=srs.to_numpy()
srs = scaler.fit_transform(temp.reshape(temp.shape[0],1))
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(np.array(seq_x))
		y.append(seq_y)
	return X,y
x,y=split_sequence(srs, 40)

In [ ]:
srs=srs[40:]


In [ ]:

# targ=srs
# print(targ.shape)
# targ=targ.reshape((targ.shape[0],1))
# X_train, X_test, y_train, y_test = train_test_split(x, targ, test_size=0.005)


In [ ]:
# X_train=np.array(X_train)

In [ ]:
# X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [ ]:
# from tensorflow.keras.models import Sequential,Model
# from tensorflow.keras.layers import LSTM,Dense,Input,Dropout,Concatenate,Flatten
# from keras.layers.merge import concatenate

# model2=Sequential()
# model2.add(LSTM(40,input_shape=(40, 1),return_sequences=True))
# model2.add(LSTM(25,return_sequences=True))
# model2.add(Flatten())
# model2.add(Dense(20,activation='relu'))
# model2.add(Dense(1,activation='relu'))


# model2.compile(loss='mean_squared_error',optimizer='adam')
# model2.fit(X_train,y_train,epochs=1,batch_size=100)

In [ ]:
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
ans=model2.predict(X_test)


In [ ]:
ans = scaler.inverse_transform(ans)


In [ ]:
y_test=scaler.inverse_transform(y_test)

In [ ]:
for i in range(len(ans)):
    print(ans[i],y_test[i])

In [ ]:
fig,ax=plt.subplots(2,figsize=(25,5))

ax[0].plot(ans[1250:],label='pred')
ax[1].plot(y_test[1250:],label='orig')
plt.legend()
plt.show()

In [ ]:
init=np.array(y_test[:40]).reshape((1,40,1))
lst=[]
print(init.shape)
for i in range(len(ans)-50):
    tp=(np.reshape(model2.predict(init),(1,)))
    lst.append(tp[0])
    init=list(init.reshape((40,)))
    init.append(lst[-1])
    init=np.array(init[1:],dtype=float).reshape((1,40,1))


In [ ]:
lst=np.array(lst)
lst=lst.reshape((lst.shape[0],1))

In [ ]:
lst=scaler.inverse_transform(lst)

In [ ]:
print(lst)

In [ ]:
y_test=scaler.inverse_transform(y_test)

In [ ]:
y_test

In [ ]:
fig,ax=plt.subplots(2,figsize=(25,5))

ax[0].plot(lst[:45],label='pred')
ax[1].plot(y_test[40:85],label='orig')
plt.legend()
plt.show()

In [ ]:
for i in range(len(ans)-90):
    print(ans[i],y_test[i+40])